In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [142]:
! chmod 600 /content/sample_data/kaggle.json

In [143]:
%cd /content/sample_data/

/content/sample_data


In [144]:
!kaggle datasets download -d nguyenngocphung/stock-prices-vn30-indexvietnam

401 - Unauthorized


In [145]:
from zipfile import ZipFile

df = pd.concat(
    [pd.read_csv(ZipFile('/content/sample_data/stock-prices-vn30-indexvietnam.zip').open(i)) for i in ZipFile('/content/sample_data/stock-prices-vn30-indexvietnam.zip').namelist()],
    ignore_index=True
)

In [146]:
df = df.drop('<Ticker>', axis = 1)

In [147]:
df.columns

Index(['<DTYYYYMMDD>', '<Open>', '<High>', '<Low>', '<Close>', '<Volume>'], dtype='object')

In [148]:
df = df.rename(columns= {'<DTYYYYMMDD>':'Date', '<Open>':'Open', '<High>': 'High', '<Low>':'Low', '<Close>':'Close', '<Volume>':'Volume'})

In [149]:
df.dtypes

Date        int64
Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object

In [150]:
df['Date'] = df['Date'].astype('object')

In [151]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [152]:
df = df.sort_values(by="Date")

In [153]:
df = df.reset_index()
df = df.drop('index', axis = 1)

In [154]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2000-07-28,16.0,16.0,16.0,1.4142,1000
1,2000-07-31,16.3,16.3,16.3,1.4407,300
2,2000-08-02,16.6,16.6,16.6,1.4673,100
3,2000-08-04,16.9,16.9,16.9,1.4938,200
4,2000-08-07,17.2,17.2,17.2,1.5203,2800
